# Data Preprocessing for Avito Using Spark

This notebook contains Data pre processing using spark as the dataset has 14 million records.

Google Cloud is the platform used.

Source of the data is https://www.kaggle.com/c/avito-demand-prediction/data

A couple of notebooks are reffered from Kaggle forums which were helpful in making data wrangling more simpler, cleaner.

https://www.kaggle.com/shivamb/in-depth-analysis-visualisations-avito

https://www.kaggle.com/kabure/extensive-eda-of-deal-probability

Thanks to both of them.



We first import libraries

As Google cloud is used and Google data-store is where the dataset is stored

In [1]:
import findspark
import pyspark
import random
import pandas as pd

In [2]:
from pyspark.sql.functions import *

In [3]:
from datetime import datetime

##### SQL context created

In [5]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

sc = SparkContext('local','example')  # if using locally
sqlContext = SQLContext(sc)

##### Data read from parquet to spark dataframe

In [65]:
df_train_active_4 = sqlContext.read.parquet("gs://dataproc-e3bd1f7b-2e29-4da6-a5c4-077c164fd32a-us-central1/avito/neat_data/train_active.parquet")

In [66]:
df_train_active_4.show(3)

+------------+------------+--------------------+---------------+--------------------+--------------------+-------+------------------+---------+--------------------+--------------------+--------+---------------+-------------------+---------+--------------------+---------------------+--------------------+-----------------------+------------------------+-----------------------+--------------------+--------------------+
|     item_id|     user_id|              region|           city|parent_category_name|       category_name|param_1|           param_2|  param_3|               title|         description|   price|item_seq_number|    activation_date|user_type|activation_date_year|activation_date_month|activation_date_date|activation_date_week_no|activation_date_week_day|parent_category_name_en|    category_name_en|           region_en|
+------------+------------+--------------------+---------------+--------------------+--------------------+-------+------------------+---------+-----------------

In [67]:
df_train_active_4.count()

14004478

In [68]:
df_train_active_4.printSchema()

root
 |-- item_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- region: string (nullable = true)
 |-- city: string (nullable = true)
 |-- parent_category_name: string (nullable = true)
 |-- category_name: string (nullable = true)
 |-- param_1: string (nullable = true)
 |-- param_2: string (nullable = true)
 |-- param_3: string (nullable = true)
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- price: double (nullable = true)
 |-- item_seq_number: double (nullable = true)
 |-- activation_date: timestamp (nullable = true)
 |-- user_type: string (nullable = true)
 |-- activation_date_year: integer (nullable = true)
 |-- activation_date_month: integer (nullable = true)
 |-- activation_date_date: integer (nullable = true)
 |-- activation_date_week_no: integer (nullable = true)
 |-- activation_date_week_day: string (nullable = true)
 |-- parent_category_name_en: string (nullable = true)
 |-- category_name_en: string (nullable = true)

In [69]:
df_train_active_4 = df_train_active_4.withColumn("param_1_len",length(df_train_active_4.param_1))

In [70]:
df_train_active_4 = df_train_active_4.withColumn("param_2_len",length(df_train_active_4.param_2))

In [71]:
df_train_active_4 = df_train_active_4.withColumn("param_3_len",length(df_train_active_4.param_3))

In [72]:
df_train_active_4 = df_train_active_4.fillna( { 'param_1_len':0, 'param_2_len':0, 'param_3_len':0 } )

In [73]:
df_train_active_4 = df_train_active_4.withColumn("param_comb_len",(df_train_active_4.param_1_len + df_train_active_4.param_2_len + df_train_active_4.param_3_len))

In [74]:
df_train_active_4 = df_train_active_4.withColumn("title_len",length(df_train_active_4.title))

In [75]:
df_train_active_4 = df_train_active_4.withColumn("description_len",length(df_train_active_4.description))

In [76]:
df_train_active_4 = df_train_active_4.fillna( { 'param_1_len':0, 'param_2_len':0, 'param_3_len':0, 'param_comb_len':0, 'title_len':0, 'description_len':0 } )

In [77]:
daymap = {"0":"1", "1":"0", "2":"0", "3":"0", "4":"0", "5":"0", "6":"1"}

In [78]:
funcWeekendMap =  udf(lambda x: daymap[x])

In [79]:
df_train_active_4 = df_train_active_4.withColumn('weekend', funcWeekendMap(col('activation_date_week_day')))

In [81]:
cols = ['region_en','category_name_en','title_len','description_len','weekend','user_type','price','param_1_len','param_2_len','param_3_len','item_seq_number']


In [82]:
df_train_active_4 = df_train_active_4.select(*cols)

In [83]:
df_train_active_4.show()

+--------------------+--------------------+---------+---------------+-------+---------+--------+-----------+-----------+-----------+---------------+
|           region_en|    category_name_en|title_len|description_len|weekend|user_type|   price|param_1_len|param_2_len|param_3_len|item_seq_number|
+--------------------+--------------------+---------+---------------+-------+---------+--------+-----------+-----------+-----------+---------------+
|Nizhny Novgorod o...| Books and magazines|       20|            117|      0|  Private|   200.0|          5|          0|          0|           20.0|
|     Voronezh oblast|Garages and Parki...|       14|            235|      0|  Private|590000.0|          6|          5|          9|           23.0|
|Nizhny Novgorod o...| Commercial property|       30|            169|      0|  Private|100000.0|          4|         18|          0|            4.0|
|       Bashkortostan|              Phones|       22|             33|      0|  Company|   750.0|         1

In [84]:
df_train_active_4.write.parquet("gs://dataproc-e3bd1f7b-2e29-4da6-a5c4-077c164fd32a-us-central1/avito/neat_data/train_active_model.parquet")

In [85]:
df_train_active_6 = sqlContext.read.parquet("gs://dataproc-e3bd1f7b-2e29-4da6-a5c4-077c164fd32a-us-central1/avito/neat_data/train_active_model.parquet")

In [86]:
df_train_active_6.printSchema()

root
 |-- region_en: string (nullable = true)
 |-- category_name_en: string (nullable = true)
 |-- title_len: integer (nullable = true)
 |-- description_len: integer (nullable = true)
 |-- weekend: string (nullable = true)
 |-- user_type: string (nullable = true)
 |-- price: double (nullable = true)
 |-- param_1_len: integer (nullable = true)
 |-- param_2_len: integer (nullable = true)
 |-- param_3_len: integer (nullable = true)
 |-- item_seq_number: double (nullable = true)

